In [1]:
import pandas as pd
import torch
import torch.nn.functional as F
import sympy as sp

In [2]:
device = torch.device("cuda:4" if torch.cuda.is_available() else "cpu")

In [8]:
import pandas as pd

df = pd.read_csv('hundo_equations.csv')

functions = []

for _, row in df.iterrows():
    formula = row['Formula']
    num_vars = row['# parameters']
    
    function_details = {
        'formula': formula,
        'num_variables': num_vars,
        'parameters': []
    }
    
    for i in range(1, num_vars + 1):
        param_name = row.get(f'p{i}_name', None)
        if pd.notna(param_name):
            function_details['parameters'].append(param_name)
    
    functions.append(function_details)

In [24]:
high = 1
low = -1
sequence_size = 1000
num_funcs = 10
max_vars = max([f['num_variables'] for f in functions])
sample_size = num_funcs*100

x = torch.linspace(high, low, sequence_size)
# x = x.unsqueeze(1).expand(sequence_size, int(sample_size/num_funcs))

param_values = []
for func in functions:
    hold = []
    for param in func['parameters']:
        p = (high - low) * torch.rand(sample_size) + low
        hold.append(p)
    params = torch.stack(hold)
    params = F.pad(params, (0, 0,0,max_vars - params.shape[0]))
    param_values.append(params)
param_values = torch.stack(param_values)

print(x.shape)
print(param_values.shape)

torch.Size([1000])
torch.Size([100, 3, 1000])


In [27]:
param_values[0, 0, 0]

tensor(-0.2514)

In [33]:
y_values = []
for f in range(len(functions)):
    formula = sp.sympify(functions[f]['formula'])
    print(formula)
    symbols = [sp.symbols(p) for p in functions[f]['parameters']]
    print(symbols)
    y = []
    for i in range(sample_size):
        var_values = {symbols[i]: param_values[f, n, i] for n in range(len(symbols))}
        print(var_values)
        evaluate = formula.subs(var_values)
        print(evaluate)
        y.append(float(evaluate))
    y_values.append(torch.stack(y))
y_values = torch.stack(y_values)
y_values.shape

ax**2 + bx + c
[a, b, c]
{a: tensor(0.9578)}
ax**2 + bx + c


TypeError: Cannot convert expression to float